In [ ]:
import showdown
import concurrent.futures
import asyncio

In [ ]:
sd = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "thisisatest12345", "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")
tasks = set()
async with asyncio.TaskGroup() as tg:
    for i in range(20):
        sd = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "PoryAI-"+str(i), "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")
        task = tg.create_task(sd.run())
        tasks.add(task)

await asyncio.gather(*tg.tasks)

#await sd.run()
# TOOD
# - Squeeze way more performance out of this
# - AAAAAAAAAAAAAAAAAAAA 
# - Dissertation?
